
# Interactive 3D Revenue Planes (Baseline vs £3/hr, £0 Sub)

This notebook renders a **single interactive 3D plot** with two planes:

- **Subscription plane**: $Z = \text{Subscription Fee} + \text{Subscription Service Fee}\times\text{Hours}$  
- **Non-Subscription plane**: $Z = \text{Non-Subscription Service Fee}\times\text{Hours}$

**Axes**  
- **X**: Hours per Month (0–100)  
- **Y**: Subscription Fee (£0–£120)  
- **Z**: Total Revenue (£)

Rotate, zoom, and hover to compare the planes. There are **no sliders** or extra plots—just this 3D view.


In [3]:
!pip install plotly

import numpy as np
import plotly.graph_objects as go

####################### Define hourly rates  ############################
subscription_fee_rate = 0.0 # £/hr for the baseline plane
non_subscription_fee_rate      = 5.0  # £/hr for the comparison plane
#########################################################################

# Grid
hours = np.linspace(0, 100, 101)    # X
subs  = np.linspace(0, 400, 121)    # Y
H, S  = np.meshgrid(hours, subs, indexing='ij')

# Planes
Z_baseline = S + subscription_fee_rate * H            # Z = S + baseline_hourly_rate*H
Z_cmp      = 0 + non_subscription_fee_rate * H                  # Z = 0 + cmp_hourly_rate*H (subscription = 0)

# Find the intersection of the two planes
# Z_baseline = Z_cmp
# S + subscription_fee_rate * H = 0 + non_subscription_fee_rate * H
# S = (non_subscription_fee_rate - subscription_fee_rate) * H

# The intersection is a line in the S-H plane, and its Z value is given by either plane equation.
# We can represent this line parametrically or by expressing S as a function of H.
# Since the grid is based on H and S, it's easiest to use S as a function of H.

H_intersection = hours # Use the same hours range as the grid
S_intersection = (non_subscription_fee_rate - subscription_fee_rate) * H_intersection
Z_intersection = S_intersection + subscription_fee_rate * H_intersection # Using the baseline plane equation

# Print the equation of the intersection line
print(f"Equation of the intersection line: Subscription Fee = (Non-Subscription Hourly Rate - Subscription Hourly Rate) * Hours")
print(f"Which simplifies to: S = ({non_subscription_fee_rate} - {subscription_fee_rate}) * H")


# Build interactive 3D figure with two surfaces
fig = go.Figure()

# Baseline plane
fig.add_trace(go.Surface(
    z=Z_baseline, x=H, y=S,
    name=f"Baseline: Z = Subscription + {subscription_fee_rate}×Hours",
    showscale=True
))

# Comparison plane (distinct opacity, no colorbar)
fig.add_trace(go.Surface(
    z=Z_cmp, x=H, y=S,
    name=f"£{non_subscription_fee_rate}/hr, £0 subscription: Z = {non_subscription_fee_rate}×Hours",
    opacity=0.7,
    showscale=False
))

# Add the intersection line
fig.add_trace(go.Scatter3d(
    x=H_intersection,
    y=S_intersection,
    z=Z_intersection,
    mode='lines',
    name='Intersection Line',
    line=dict(color='red', width=5)
))

# Add a vertical plane for the intersection line
# Create a grid for the vertical plane based on the intersection line
S_plane = np.linspace(0, 400, 121) # Use the same S range as the main grid
H_plane, S_plane_grid = np.meshgrid(H_intersection, S_plane, indexing='ij')

# Calculate the Z values for the vertical plane.
# The Z value at the intersection is given by Z_intersection.
# We need to extend this vertically. The vertical plane has the same S_intersection value for each H.
# However, for the surface plot, we need Z values for the entire S_plane_grid.
# The intersection line is where S = (non_subscription_fee_rate - subscription_fee_rate) * H
# We can create a surface where S is fixed at S_intersection for each H and Z varies across the full range.
# A simpler way is to create a plane that is vertical to the H-S plane at the intersection line.
# This vertical plane has a fixed relationship between S and H defined by the intersection line S_intersection = (non_subscription_fee_rate - subscription_fee_rate) * H_intersection.
# For a given H, the S value is fixed by the intersection line equation, and the Z value spans the range of the plot.

# Let's redefine the approach for the vertical plane:
# The vertical plane will have x = H_intersection, y = S_intersection, and z varying from 0 to max Z.
# Since S_intersection varies with H, this will be a curved vertical plane.

# Create a grid for the vertical plane
hours_plane = np.linspace(0, 100, 101)
# Calculate the corresponding subscription fees for the intersection line
subs_plane = (non_subscription_fee_rate - subscription_fee_rate) * hours_plane
# Create a Z range that covers the plot's Z axis
z_plane = np.linspace(0, np.max([Z_baseline, Z_cmp]), 101)

# Create the surface data. For each point (h, s_intersect) on the intersection line in the H-S plane,
# the Z value can be any value in z_plane.
# We need to create a grid where H and S are from the intersection line and Z spans the required range.
# This is not a standard surface where Z is a function of H and S.
# Instead, we can define the plane parametrically or by fixing the relationship between H and S.

# Let's try creating a surface with H and Z as independent variables, and S determined by the intersection equation.
H_surf, Z_surf = np.meshgrid(hours_plane, z_plane, indexing='ij')
S_surf = (non_subscription_fee_rate - subscription_fee_rate) * H_surf

# Filter out points where S_surf is outside the relevant S range (0-400)
valid_indices = (S_surf >= 0) & (S_surf <= 400)
H_surf_filtered = H_surf[valid_indices]
S_surf_filtered = S_surf[valid_indices]
Z_surf_filtered = Z_surf[valid_indices]

# Reshape for go.Surface (needs 2D arrays)
# This approach with filtering makes reshaping difficult for go.Surface.
# Let's reconsider the definition of the vertical plane.

# A vertical plane at the intersection line means that for any point (H, S, Z) on this plane,
# S = (non_subscription_fee_rate - subscription_fee_rate) * H holds.
# We can define this surface using a parametric approach or by specifying the coordinates directly.

# Let's use the H_intersection and S_intersection from the line calculation.
# We need a Z range for each point on the intersection line.
num_points = len(H_intersection)
num_z_levels = 101 # Number of points in the vertical direction

# Create arrays for the surface
H_plane_surf = np.repeat(H_intersection, num_z_levels).reshape(-1, num_z_levels)
S_plane_surf = np.repeat(S_intersection, num_z_levels).reshape(-1, num_z_levels)
Z_plane_surf = np.tile(np.linspace(0, np.max([Z_baseline, Z_cmp]), num_z_levels), num_points).reshape(-1, num_z_levels)

fig.add_trace(go.Surface(
    x=H_plane_surf,
    y=S_plane_surf,
    z=Z_plane_surf,
    name='Intersection Plane',
    opacity=0.5,
    colorscale=[[0, 'red'], [1, 'red']], # Solid red color
    showscale=False
))


fig.update_layout(
    title="Total Revenue: Baseline vs £3/hr (0 subscription)",
    scene=dict(
        xaxis_title="Hours per Month (0–100)",
        yaxis_title="Subscription Fee (£0–£120)",
        zaxis_title="Total Revenue (£)",
        camera=dict(
            eye=dict(x=2, y=2, z=2) # Adjust eye values to control zoom
        )
    ),
    legend=dict(
        y=0.95, x=0.01, bgcolor="rgba(255,255,255,0.6)"
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

fig.show()

Equation of the intersection line: Subscription Fee = (Non-Subscription Hourly Rate - Subscription Hourly Rate) * Hours
Which simplifies to: S = (5.0 - 0.0) * H
